# Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Reading the Files

In [ ]:
# Reading Train File
train_df = pd.read_csv("../input/dont-overfit-ii/train.csv")
#train_df.head()
train_df = train_df.drop("id", axis=1)
# Create a data with all columns except target
X_train_not_scaled = train_df.drop("target", axis=1)

# Create a target. labels dataset
y_train = (train_df[["target"]])

# Reading Test File
test_df = pd.read_csv("../input/dont-overfit-ii/test.csv")
#test_df.head() 
X_test_not_scaled = test_df.drop(columns=['id'])

In [ ]:
train_df.shape
X_test_not_scaled.shape

# Missing Values

In [ ]:
train_df.isnull().values.any()

In [ ]:
X_test_not_scaled.isnull().values.any()

In [ ]:
#train_df.isnull().sum().sort_values(ascending = False).head(2)

In [ ]:
#X_test_not_scaled.isnull().sum().sort_values(ascending = False).head(2)

# Check for Non Numaric values

In [ ]:
train_df.applymap(np.isreal).values.all()

In [ ]:
X_test_not_scaled.applymap(np.isreal).values.all()

# Standard Scaling

train data

In [ ]:
from sklearn.preprocessing import StandardScaler

#Scaling Numerical columns
std = StandardScaler()
X_train = std.fit_transform(X_train_not_scaled)
X_train = pd.DataFrame(X_train)
#X_train = X_train_not_scaled.merge(X_train,left_index=True,right_index=True,how = "left")
X_train.shape

test data

In [ ]:
#Scaling Numerical columns
std = StandardScaler()
X_test = std.fit_transform(X_test_not_scaled)
X_test = pd.DataFrame(X_test)
#X_test = X_test_not_scaled.merge(X_test,left_index=True,right_index=True,how = "left")
X_test.shape

In [ ]:
y_train.value_counts()

# GridSearch CV and Modeling

here i have used grid search to find the best scores for the logistic regression model:
* i have used the **"liblinear"** solver as its very suitable at small data set as we have at the training data 
* for the grid parameters i choosed:
  * {"**class_wight**"} : as the target here at the data is **not equally distributed** between 0,1 o records are bigger than 1 records 2.7 times
  * {"**penalty**"} : to ckhoose the best **norm-method** for the solver in optimization of the cost function.
  * {"**C**"} : to control the REGULARIZATION to avoid the OVERFITTING of the model as **C equals the reciprocal of the LAMBDA hyper parameter**. 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Find best hyperparameters (roc_auc)
random_state = 42
log_clf = LogisticRegression(solver='liblinear',random_state = random_state)
param_grid = {'class_weight' : ['balanced', None], 
                'penalty' : ['l2','l1'],  
                'C' : [0.001, 0.01, 0.1, 1, 10, 100]}

grid = GridSearchCV(estimator = log_clf, param_grid = param_grid , scoring = 'roc_auc', verbose = 1, n_jobs = -1, cv = 20)

grid.fit(X_train,y_train)

print("Best Score:" + str(grid.best_score_))
print("Best Parameters: " + str(grid.best_params_))

best_parameters = grid.best_params_

In [ ]:
log_clf = LogisticRegression(solver='liblinear',random_state = random_state,**best_parameters)
log_clf.fit(X_train,y_train)

# AUCROC "just for showing"

In [ ]:
from sklearn import metrics
y_pred_proba = log_clf.predict_proba(X_train)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_train,  y_pred_proba)
auc = metrics.roc_auc_score(y_train, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

# Test Prediction

In [ ]:
y_pred_0_1 = log_clf.predict(X_test)
y_pred_0_1

In [ ]:
y_test_predicted_proba = log_clf.predict_proba(X_test)[::,1]
y_test_predicted_proba

# Preparing Submission files

In [ ]:
test_df['target'] = y_pred_0_1

In [ ]:
test_df[['id', 'target']].to_csv('/kaggle/working/submission.csv', index=False)

i don't know why are the compitition scores not improving in significant way !! i have tried many things in grid search of the model and data preparation and this notebook is the best scores of my work which didn't vary significantly!!, this may be due any change in the data sets from KAGGLE side and i wish to know the reason !!!